# Chapter 5, Report

Ebook, A Hands-On Guide to Biomechanics Data Analysis with Python and AI  
Author, Dr. Hossein Mokhtarzadeh  
Powered by PoseIQ

Goal, compute key metrics, save figures, and export a simple report.  
Click Runtime, Run all.


[Open in Colab](YOUR_GITHUB_COLAB_URL_HERE)

## Install packages

In [ ]:
!pip -q install ezc3d pandas scipy matplotlib

## Settings, you can change these

In [ ]:
body_mass_kg = 75.0  # set your subject mass if you want GRF in body weights
marker_cutoff_hz = 6.0
force_cutoff_hz  = 20.0
report_dir = "report_outputs"


## Download sample data from c3d.org, with fallback

In [ ]:
import os, urllib.request, zipfile

def safe_get(url, out_path):
    try:
        urllib.request.urlretrieve(url, out_path)
        return True
    except Exception as e:
        print("Download failed:", e)
        return False

os.makedirs("sample_data", exist_ok=True)
zpath = "sample_data.zip"
ok = safe_get("https://c3d.org/data/Sample00.zip", zpath) or safe_get("https://www.c3d.org/data/Sample00.zip", zpath) \
     or safe_get("https://c3d.org/data/Sample01.zip", zpath) or safe_get("https://www.c3d.org/data/Sample01.zip", zpath)

if ok:
    with zipfile.ZipFile(zpath, 'r') as zf:
        zf.extractall("sample_data")
    print("Extracted sample_data")
else:
    print("Could not download sample data. Upload your .c3d file into sample_data")


## Load first C3D file and parse time, markers, and analogs

In [ ]:
import os, ezc3d, numpy as np, pandas as pd
from scipy.signal import butter, filtfilt, savgol_filter

# Find a C3D file recursively
c3d_files = []
for root, dirs, files in os.walk("sample_data"):
    for f in files:
        if f.lower().endswith(".c3d"):
            c3d_files.append(os.path.join(root, f))
if not c3d_files:
    raise FileNotFoundError("No .c3d files found. Upload a .c3d file to sample_data.")

c3d_path = c3d_files[0]
c3d = ezc3d.c3d(c3d_path)

points = c3d["data"]["points"]                 # (4, n_markers, n_frames)
n_markers = points.shape[1]
n_frames  = points.shape[2]

# Rates and time
if "POINT" in c3d["parameters"] and "RATE" in c3d["parameters"]["POINT"]:
    point_rate = float(c3d["parameters"]["POINT"]["RATE"]["value"][0])
else:
    point_rate = float(c3d["header"]["points"]["frame_rate"])
time = np.arange(n_frames) / point_rate

# Marker labels
marker_labels = list(c3d["parameters"]["POINT"]["LABELS"]["value"]) if "POINT" in c3d["parameters"] and "LABELS" in c3d["parameters"]["POINT"] else [f"M{i}" for i in range(n_markers)]

# Choose heel like marker if present, else first
preferred = ["RHEE","RHEEL","HEEL","R_Heel","RHEE1","RHEEL1"]
m_idx = 0
for name in preferred:
    if name in marker_labels:
        m_idx = marker_labels.index(name)
        break

# Marker in meters and filtered
xyz = points[:3, m_idx, :].T / 1000.0
def lowpass(sig, fs_hz, cutoff_hz=6.0, order=4, axis=0):
    nyq = 0.5 * fs_hz
    b, a = butter(order, cutoff_hz/nyq, btype="low")
    return filtfilt(b, a, sig, axis=axis)

xyz_f = lowpass(xyz, fs_hz=point_rate, cutoff_hz=marker_cutoff_hz, order=4, axis=0)

# Analogs
analogs = c3d["data"]["analogs"]               # (n_subframes, n_analogs, n_frames)
has_analogs = isinstance(analogs, np.ndarray) and analogs.size > 0
analog_rate = None
analog_labels = []
if "ANALOG" in c3d["parameters"]:
    parA = c3d["parameters"]["ANALOG"]
    if "RATE" in parA:
        analog_rate = float(parA["RATE"]["value"][0])
    for k, v in parA.items():
        if k.startswith("LABELS"):
            analog_labels.extend(list(v["value"]))

# Choose an Fz like channel if available
force_ch = 0
if analog_labels:
    cand = [i for i, lab in enumerate(analog_labels) if isinstance(lab, str) and lab.lower().startswith("fz")]
    if not cand:
        cand = [i for i, lab in enumerate(analog_labels) if isinstance(lab, str) and "force" in lab.lower()]
    if cand:
        force_ch = cand[0]

# Build a continuous analog series for that channel
if has_analogs and analog_rate:
    an2 = analogs.transpose(1,2,0).reshape(analogs.shape[1], -1)
    force_raw = an2[force_ch, :].astype(float)
    # Filter force
    force_f = lowpass(force_raw, fs_hz=analog_rate, cutoff_hz=force_cutoff_hz, order=4, axis=0)
else:
    analog_rate = None
    force_raw = None
    force_f = None

# Prepare DataFrame aligned to marker frames for reporting
df = pd.DataFrame({
    "Time": time,
    "Heel_X": xyz_f[:,0],
    "Heel_Y": xyz_f[:,1],
    "Heel_Z": xyz_f[:,2],
})

# If possible, decimate force to marker frames for convenient plotting together
if force_f is not None and analog_rate is not None:
    step = max(1, int(round(analog_rate/point_rate)))
    df["Fz_like"] = force_f[::step][:len(df)]
else:
    df["Fz_like"] = np.nan

print("DataFrame columns:", list(df.columns))


## Detect contacts and build cycles

In [ ]:
import numpy as np

if df["Fz_like"].notna().sum() > 0:
    y = df["Fz_like"].values.astype(float)
    thr = 0.05 * np.nanmax(np.abs(y)) if np.nanmax(np.abs(y)) > 0 else 0.0
    contact = y > thr
    onsets = np.where((contact[1:] & ~contact[:-1]))[0] + 1
    offsets = np.where((~contact[1:] & contact[:-1]))[0] + 1
    heel_strikes_t = df["Time"].values[onsets]
    toe_offs_t     = df["Time"].values[offsets]
else:
    y = None
    thr = None
    heel_strikes_t = np.array([])
    toe_offs_t = np.array([])

print("Heel strikes s:", np.round(heel_strikes_t, 3)[:10])
print("Toe offs s:", np.round(toe_offs_t, 3)[:10])


## Compute core metrics

In [ ]:
import numpy as np, pandas as pd

metrics = {}

# Peak GRF in body weights if mass known
if y is not None and np.nanmax(np.abs(y)) > 0:
    peak_N = float(np.nanmax(y))
    metrics["Peak_force_raw"] = peak_N
    if body_mass_kg and body_mass_kg > 0:
        metrics["Peak_force_BW"] = peak_N / (body_mass_kg * 9.81)
else:
    metrics["Peak_force_raw"] = np.nan
    metrics["Peak_force_BW"] = np.nan

# Stride times and cadence
if heel_strikes_t.size > 1:
    stride_times = np.diff(heel_strikes_t)
    metrics["Stride_time_mean_s"] = float(np.mean(stride_times))
    metrics["Cadence_spm"] = 60.0 / metrics["Stride_time_mean_s"]
else:
    metrics["Stride_time_mean_s"] = np.nan
    metrics["Cadence_spm"] = np.nan

# Step length from Heel_X
step_lengths = []
if heel_strikes_t.size > 1:
    for t0, t1 in zip(heel_strikes_t[:-1], heel_strikes_t[1:]):
        seg = df[(df["Time"] >= t0) & (df["Time"] < t1)]
        if len(seg) > 1:
            step_lengths.append(float(seg["Heel_X"].iloc[-1] - seg["Heel_X"].iloc[0]))
metrics["Step_length_mean_m"] = float(np.mean(step_lengths)) if step_lengths else np.nan

# Contact time per step
contact_times = []
if heel_strikes_t.size > 0 and toe_offs_t.size > 0:
    for hs in heel_strikes_t:
        after = toe_offs_t[toe_offs_t > hs]
        if after.size > 0:
            contact_times.append(float(after[0] - hs))
metrics["Contact_time_mean_s"] = float(np.mean(contact_times)) if contact_times else np.nan

# Loading rate estimate, peak slope over first 10 percent of stance
def loading_rate(sig, fs_hz):
    n = len(sig)
    if n < 3:
        return np.nan
    # simple max finite difference per second
    ds = np.diff(sig) * fs_hz
    return float(np.nanmax(ds))

if y is not None and heel_strikes_t.size > 0 and analog_rate is not None:
    lrates = []
    t = df["Time"].values
    for hs in heel_strikes_t:
        # 10 percent of median stride or 0.2 s
        win = 0.2
        m = (t >= hs) & (t < hs + win)
        seg = df.loc[m, "Fz_like"].values
        if seg.size > 5:
            lrates.append(loading_rate(seg, point_rate))
    metrics["Loading_rate_max"] = float(np.nanmax(lrates)) if lrates else np.nan
else:
    metrics["Loading_rate_max"] = np.nan

metrics_df = pd.DataFrame([metrics])
metrics_df


## Figures, save plots to report_outputs

In [ ]:
import os, matplotlib.pyplot as plt, numpy as np
os.makedirs(report_dir, exist_ok=True)

# Plot force like signal if available
if df["Fz_like"].notna().sum() > 0:
    plt.figure(figsize=(10,4))
    plt.plot(df["Time"], df["Fz_like"])
    plt.xlabel("Time s")
    plt.ylabel("Force like")
    plt.title("Force like signal over time")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f"{report_dir}/force_time.png", dpi=150)
    plt.show()

# Plot Heel_Z with Savitzky Golay smoothing
df["Heel_Z_sg"] = savgol_filter(df["Heel_Z"].values, window_length=11, polyorder=3)
plt.figure(figsize=(10,4))
plt.plot(df["Time"], df["Heel_Z"], label="Heel_Z raw")
plt.plot(df["Time"], df["Heel_Z_sg"], label="Heel_Z sg")
plt.xlabel("Time s")
plt.ylabel("Heel Z m")
plt.title("Marker vertical position")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(f"{report_dir}/heel_z.png", dpi=150)
plt.show()

# Cycle normalised mean force like
def norm_cycle(sig, t, t0, t1, n=101):
    m = (t >= t0) & (t < t1)
    if m.sum() < 2:
        return None
    seg = sig[m]
    xp = np.linspace(0, 1, seg.size)
    xnew = np.linspace(0, 1, n)
    return np.interp(xnew, xp, seg)

cycles = []
if df["Fz_like"].notna().sum() > 0 and heel_strikes_t.size > 1:
    t = df["Time"].values
    for a, b in zip(heel_strikes_t[:-1], heel_strikes_t[1:]):
        c = norm_cycle(df["Fz_like"].values, t, a, b, n=101)
        if c is not None:
            cycles.append(c)

if cycles:
    import numpy as np
    arr = np.vstack(cycles)
    mean_cycle = np.nanmean(arr, axis=0)
    xpc = np.linspace(0, 100, arr.shape[1])
    plt.figure(figsize=(8,4))
    for i in range(min(8, arr.shape[0])):
        plt.plot(xpc, arr[i], alpha=0.4)
    plt.plot(xpc, mean_cycle, linewidth=2)
    plt.xlabel("Gait cycle percent")
    plt.ylabel("Force like")
    plt.title("Normalised cycles and mean")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f"{report_dir}/force_cycles.png", dpi=150)
    plt.show()


## Export, CSVs and a simple HTML report

In [ ]:
import pandas as pd, os

# Save metrics and a trimmed data sample
metrics_path = f"{report_dir}/metrics.csv"
df_path = f"{report_dir}/timeseries_sample.csv"

metrics_df.to_csv(metrics_path, index=False)
df[["Time","Heel_X","Heel_Y","Heel_Z","Fz_like","Heel_Z_sg"]].head(500).to_csv(df_path, index=False)

# Simple HTML report with links to images and tables
html_path = f"{report_dir}/report.html"
with open(html_path, "w", encoding="utf-8") as f:
    f.write("<h1>Chapter 5 Report</h1>")
    f.write("<p>Ebook, A Hands-On Guide to Biomechanics Data Analysis with Python and AI</p>")
    f.write("<h2>Metrics</h2>")
    f.write(pd.read_csv(metrics_path).to_html(index=False))
    if os.path.exists(f"{report_dir}/force_time.png"):
        f.write("<h2>Force over time</h2><img src='force_time.png' width='700'>")
    f.write("<h2>Heel Z</h2><img src='heel_z.png' width='700'>")
    if os.path.exists(f"{report_dir}/force_cycles.png"):
        f.write("<h2>Normalised force cycles</h2><img src='force_cycles.png' width='700'>")
    f.write("<p>Data sample, see timeseries_sample.csv</p>")

print("Saved,")
print(metrics_path)
print(df_path)
print(html_path)


## Summary

You computed peak force, cadence, step length, contact time, and loading rate.  
You saved figures and exported a simple HTML report plus CSV files.  
Readers can change body mass, cutoff frequencies, and output directory.
